In [2]:
import cftime
import glob
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
from scipy import stats

import mission_tools

%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
mission_list = glob.glob('../microSWIFT_data/final_dataset/mission_*.nc')

wave_heights_all = []
wave_x_locs_all = []
wave_y_locs_all = []
wave_times_all = []
mission_number = []

for mission_nc in mission_list:
    mission_dataset = nc.Dataset(mission_nc, mode='r')

    x_locations = mission_dataset['xFRF'][:]
    y_locations = mission_dataset['yFRF'][:]
    eta = mission_dataset['sea_surface_elevation'][:]
    time = mission_dataset['time'][:]

    wave_heights, \
    wave_x_locs, \
    wave_y_locs, \
    wave_times = mission_tools.compute_individual_waves_with_time(x_locations, y_locations,
                                                                  eta, time,
                                                                  single_trajectory=False )
    
    wave_heights_all.append(wave_heights)
    wave_x_locs_all.append(wave_x_locs)
    wave_y_locs_all.append(wave_y_locs)
    wave_times_all.append(cftime.num2pydate(wave_times, units=mission_dataset['time'].units,
                          calendar=mission_dataset['time'].calendar))
    
    mission_number.append(int(mission_nc[41:-3]) * np.ones(len(wave_heights)))

wave_heights_all = np.concatenate(wave_heights_all).flatten()
wave_x_locs_all = np.concatenate(wave_x_locs_all).flatten()
wave_y_locs_all = np.concatenate(wave_y_locs_all).flatten()
wave_times_all = np.concatenate(wave_times_all).flatten()
mission_number = np.concatenate(mission_number).flatten()

/opt/homebrew/Caskroom/mambaforge/base/envs/dunex-venv/lib/python3.11/site-packages/numpy/ma/core.py:5305: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
/opt/homebrew/Caskroom/mambaforge/base/envs/dunex-venv/lib/python3.11/site-packages/numpy/core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [25]:
# Create a pandas dataframe from all wave data
wave_df = pd.DataFrame(wave_times_all, columns=['time'])
wave_df['mission number'] = mission_number
wave_df['height [m]'] = wave_heights_all
wave_df['cross shore location [m]'] = wave_x_locs_all
wave_df['along shore location [m]'] = wave_y_locs_all

wave_df

,time,mission number,height [m],cross shore location [m],along shore location [m]
0,2021-10-29 14:29:04.541608,78.0,1.634624,365.133427,549.829861
1,2021-10-29 14:29:11.374914,78.0,0.818419,364.549602,553.360727
2,2021-10-29 14:29:14.499902,78.0,0.395207,362.869111,554.889660
3,2021-10-29 14:29:17.749889,78.0,0.440129,363.308244,556.838904
4,2021-10-29 14:29:22.583203,78.0,1.093854,364.066896,558.825392
...,...,...,...,...,...
116524,2021-10-22 14:47:02.991108,51.0,0.006356,97.951538,373.158218
116525,2021-10-22 14:48:22.865788,51.0,0.004740,97.952482,373.160959
116526,2021-10-22 14:49:40.865476,51.0,0.004470,98.011941,373.333641
116527,2021-10-22 14:49:47.448784,51.0,0.003421,98.011941,373.333641


In [32]:
wave_df.to_csv('../microSWIFT_data/wave_realizations_df.csv')